In [45]:
%pip install deepface
%pip install opencv-python
%pip install ultralytics
%pip install sqlalchemy


In [46]:
from deepface import DeepFace
from ultralytics import YOLO
import cv2
import uuid
import os
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [82]:

def crowd_analysis(video_input):
  # Create 'detected' directory if it doesn't exist
  output_dir = "detected"
  os.makedirs(output_dir, exist_ok=True)

  # SQLAlchemy setup
  Base = declarative_base()

  class DetectedPerson(Base):
      __tablename__ = 'table_detected'

      id = Column(Integer, primary_key=True, autoincrement=True)
      age = Column(Integer)
      gender = Column(String)
      race = Column(String)
      emotion = Column(String)
      uuid = Column(String)
      detection_time = Column(DateTime)

  # Create an SQLite database
  engine = create_engine('sqlite:///detected_people.db')
  Base.metadata.create_all(engine)

  Session = sessionmaker(bind=engine)
  session = Session()

  # Load the YOLO model
  model = YOLO("yolov8n-person.pt")

  # Open video file or capture device
  cap = cv2.VideoCapture(video_input)

  # Process the video frame by frame
  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      # Detect people in the frame
      results = model(frame)[0]

      # Process each detected person
      for result in results.boxes.data.tolist():
          x1, y1, x2, y2, confidence_score, class_id = result
          x1 = int(x1)
          y1 = int(y1)
          x2 = int(x2)
          y2 = int(y2)

          # Ensure the coordinates are within the frame bounds
          if x1 < 0: x1 = 0
          if y1 < 0: y1 = 0
          if x2 > frame.shape[1]: x2 = frame.shape[1]
          if y2 > frame.shape[0]: y2 = frame.shape[0]

          # Crop the person from the frame
          cropped_person = frame[y1:y2, x1:x2]
          # Generate a unique filename using uuid
          image_uuid = uuid.uuid4().hex
          filename = os.path.join(output_dir, f"{image_uuid}.jpg")

          # Save the cropped face image
          cv2.imwrite(filename, cropped_person)


          try:
              # Analyze the cropped person
              objs = DeepFace.analyze(
                  img_path=cropped_person,
                  actions=['age', 'gender', 'race', 'emotion'],
              )

              # Extract and store only the dominant traits
              for obj in objs:

                  # Create the dictionary with the required information
                  person_info = {
                      "age": obj['age'],
                      "gender": obj['dominant_gender'],
                      "race": obj['dominant_race'],
                      "emotion": obj['dominant_emotion'],
                      "uuid": image_uuid,
                      "detection_time": datetime.now()
                  }

                  # Save to the database
                  detected_person = DetectedPerson(
                      age=person_info['age'],
                      gender=person_info['gender'],
                      race=person_info['race'],
                      emotion=person_info['emotion'],
                      uuid=person_info['uuid'],
                      detection_time=person_info['detection_time']
                  )
                  session.add(detected_person)
                  session.commit()
                  print(person_info)
          except ValueError as e:
              print(f"Error: {e} - Skipping to next detected person.")

  cap.release()
  print("Video processing completed.")
cf=crowd_analysis("vidtest.mp4")



0: 736x1280 3 persons, 12.9ms
Speed: 4.3ms preprocess, 12.9ms inference, 1.9ms postprocess per image at shape (1, 3, 736, 1280)
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.

0: 736x1280 3 persons, 12.2ms
Speed: 6.9ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 736, 1280)
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  6.73it/s]

{'age': 25, 'gender': 'Man', 'race': 'asian', 'emotion': 'fear', 'uuid': '9be5444e12b4486380a3abfa7fa6fa4a', 'detection_time': datetime.datetime(2024, 8, 10, 19, 55, 27, 83514)}

0: 736x1280 2 persons, 11.0ms
Speed: 6.2ms preprocess, 11.0ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280)


Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.

0: 736x1280 2 persons, 11.6ms
Speed: 4.5ms preprocess, 11.6ms inference, 2.4ms postprocess per image at shape (1, 3, 736, 1280)
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.

0: 736x1280 2 persons, 10.2ms
Speed: 4.8ms preprocess, 10.2ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.98it/s]

...

{'age': 25, 'gender': 'Man', 'race': 'asian', 'emotion': 'neutral', 'uuid': '324fd3b34c91479b9791bfa117c23da8', 'detection_time': datetime.datetime(2024, 8, 10, 19, 55, 31, 737542)}
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.

0: 736x1280 3 persons, 12.2ms
Speed: 4.3ms preprocess, 12.2ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)


Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.71it/s]


{'age': 28, 'gender': 'Man', 'race': 'asian', 'emotion': 'neutral', 'uuid': '4cedf73e22734a0496beeb870ba5b584', 'detection_time': datetime.datetime(2024, 8, 10, 19, 55, 32, 924588)}
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. - Skipping to next detected person.

0: 736x1280 3 persons, 12.3ms
Speed: 4.8ms preprocess, 12.3ms inference, 1.3ms postprocess per image at shape (1, 3, 736, 1280)


Action: gender:  25%|██▌       | 1/4 [00:00<00:00,  5.06it/s]


KeyboardInterrupt: 

In [66]:
import base64
import requests

def describe_image(base64_image: str):
    api_key = "Enter API KEY"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant that describes images."
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe this image:"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 50
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

# Example usage
base64_image = "your_base64_image_string_here"  # Replace this with your actual base64 string
description = describe_image(base64_image)
print(description)

{'error': {'message': 'Invalid base64 image_url.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_base64'}}


In [86]:
# Make sure to define or import the ComparePerson class at the beginning of the script
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

# Define the SQLAlchemy Base
Base = declarative_base()

# Define the ComparePerson model
class ComparePerson(Base):
    __tablename__ = 'table_compare'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    uuid = Column(String)

# Setup the database engine and session
engine = create_engine('sqlite:///detected_people.db')
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Function to insert a new person into the table_compare
def insert_person(name, uuid_value):
    # Create a new instance of ComparePerson with the provided name and UUID
    new_person = ComparePerson(name=name, uuid=uuid_value)

    # Add the record to the session
    session.add(new_person)

    # Commit the session to insert the record into the database
    session.commit()

    print(f"Record inserted successfully for {name} with UUID {uuid_value}.")

# Example usage to insert John Doe with UUID 'img_925.jpg'
insert_person("Alex", "Alex.png")


Record inserted successfully for Alex with UUID Alex.png.


In [91]:
def perform_face_recognition(name):
    detect_folder = "detected"

    # Iterate over the images in the detect directory
    for image_file in os.listdir(detect_folder):
        image_path = os.path.join(detect_folder, image_file)

        try:
            # Perform face recognition with enforce_detection set to False
            results = DeepFace.find(
                img_path=image_path,
                db_path="detected",
                enforce_detection=False,
                silent=True  # Suppress DeepFace output
            )

            # Check if any matches are found
            if len(results) > 0:
                # Get the UUID of the matched image
                matched_uuid = os.path.splitext(image_file)[0]

                # Retrieve information from table_detected
                detected_person = session.query(DetectedPerson).filter_by(uuid=matched_uuid).first()

                if detected_person:
                    # Add the matched person to the table_compare
                    add_person_to_compare(name, matched_uuid)
                    # Only print the match found and the matched person's details
                    print(f"Match found for {name}: UUID {matched_uuid}")
                    print(f"Record for UUID {matched_uuid}:")
                    print(f"Age: {detected_person.age}")
                    print(f"Gender: {detected_person.gender}")
                    print(f"Race: {detected_person.race}")
                    print(f"Emotion: {detected_person.emotion}")
                    print(f"Detection Time: {detected_person.detection_time}")

        except Exception as e:
            # You can log this exception if needed or pass
            pass

# Example usage
perform_face_recognition("Alex")

print("Face recognition completed.")

Match found for Alex: UUID be8e157dec8e4136aec5f959c2ad3da5
Record for UUID be8e157dec8e4136aec5f959c2ad3da5:
Age: 25
Gender: Man
Race: asian
Emotion: neutral
Detection Time: 2024-08-10 19:55:30.264512
Match found for Alex: UUID ce2a45e722c34213ab7dd6e7e6f044d4
Record for UUID ce2a45e722c34213ab7dd6e7e6f044d4:
Age: 21
Gender: Man
Race: indian
Emotion: neutral
Detection Time: 2024-08-10 19:55:28.417400
Match found for Alex: UUID 9be5444e12b4486380a3abfa7fa6fa4a
Record for UUID 9be5444e12b4486380a3abfa7fa6fa4a:
Age: 25
Gender: Man
Race: asian
Emotion: fear
Detection Time: 2024-08-10 19:55:27.083514
Match found for Alex: UUID 324fd3b34c91479b9791bfa117c23da8
Record for UUID 324fd3b34c91479b9791bfa117c23da8:
Age: 25
Gender: Man
Race: asian
Emotion: neutral
Detection Time: 2024-08-10 19:55:31.737542
Match found for Alex: UUID 4cedf73e22734a0496beeb870ba5b584
Record for UUID 4cedf73e22734a0496beeb870ba5b584:
Age: 28
Gender: Man
Race: asian
Emotion: neutral
Detection Time: 2024-08-10 19:55:32